In [30]:
import ollama
from langchain_ollama import OllamaLLM
from langchain.chains import create_retrieval_chain  
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain.prompts import PromptTemplate

In [31]:
model_name = "zephyr:7b-beta-q4_K_S"
llm = OllamaLLM(model=model_name, max_tokens=200) 

In [32]:
persist_directory = "../model/taxadvisordb_v1-0"

In [33]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [34]:
vector_db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [35]:
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

In [36]:
docs = retriever.invoke("Can failing to submit an income tax return lead to prosecution?")
print(docs)

[Document(id='3eebcdde-8f6f-447c-a3c1-73a28edfc6f2', metadata={'keywords': 'taxable income, income tax return, file taxes, non-filing, tax consequences, penalties, IRS', 'source': 'ITR_FAQs.csv'}, page_content="Query: What will happen when I have taxable income but do not File my Income Tax Return?\nSolution: Having taxable income and not filing an income tax return can put you in trouble with the Income Tax Authorities. Be prepared to welcome notices from the Income Tax Department along with fines and penalties. The consequences of non-filing of return are:\n\nLate filing Fee u/s 234F shall be levied if the return is not filed on or before the due date. The quantum of fees shall vary according to the amount of your income and return filing date. The maximum fee levied is Rs. 5000/-.\nNotice u/s 142(1), which requires you to file your return if you have not filed it before the due date. if you fail to comply with this notice, then it may result in\n(a) Best Judgment Assessment u/s 144 

In [14]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [17]:
prompt_template = PromptTemplate(
    input_variables=["context", "query"],
    template=(
        "You are a tax expert. Use the following information to answer the user's question:\n\n"
        "Context:\n{context}\n\n"
        "Question: {query}\n\n"
        "Answer:"
    ),
)

In [37]:
import asyncio

async def get_answer(query):
    retriever = vector_db.as_retriever(search_kwargs={"k": 2})  # Fetch fewer docs
    docs = await asyncio.to_thread(retriever.invoke, query)  # Async retrieval

    context = "\n\n".join([doc.page_content for doc in docs])
    prompt = f"You are a tax expert. Use the following information to answer the user's question:\n\n{context}\n\nQuery: {query}\nAnswer:"

    # Stream response from LLM
    async for chunk in llm.astream(prompt):  
        print(chunk, end="", flush=True)

In [40]:
query = "is aadhar neccessery for filing itr"
await get_answer(query)

As per the current tax laws, it is mandatory to link your PAN with Aadhaar while filing your ITR (Income Tax Return). This means that you need to provide your Aadhaar number while filling out your ITR form. However, if you have not yet linked your PAN and Aadhaar, you can still file your return, but you will have limited access on the portal. It is advisable to link your PAN with Aadhaar as soon as possible to avoid any inconvenience during the ITR filing process. So, in short, Aadhaar is necessary for filing ITR, as it serves as an important identification document required by the tax authorities.